# Webscraping Newspaper - ETL

## Vorbereitung

In [2]:
import os
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
import datetime as dt
import itertools
import glob as glob
from time import time
import re
from langdetect import detect
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Daten laden und selektieren  
**paper_list** umfasst News-Portale (4), die in die Analyse aufgenommen werden sollen  
**date_list** umfast alle Daten vom 03.03.2021 bis zum 31.01.2022 (335), die analysiert werden sollen  

In [3]:
INPUT_BASE_PATH = os.path.join("..","input")
INPUT_PATH = os.path.join(INPUT_BASE_PATH,"data-lake")
SQL_PATH = os.path.join("..", "output", "dwh.sqlite3")

In [4]:
connection = sqlite3.connect(SQL_PATH)

In [5]:
# paper_list umfasst alle für die Analyse ausgewählte News-Portale.
paper_list = ["abendblatt", "sz", "handelsblatt", "cnn", "bbc", "economist", "economist "]

In [6]:
# Anzahl News-Portale:
number_of_newspapers = len(paper_list)
number_of_newspapers

7

In [7]:
start_date = dt.date.fromisoformat("2021-03-02")
start_date

datetime.date(2021, 3, 2)

In [8]:
day = dt.timedelta(days=1)

In [9]:
#date_list: Daten vom 03.03.2021 bis zum 31.01.2022 erzeugt und als Liste gespeichert

date_list= []
for i in range(335):
    start_date += day
    date_list.append(start_date)
#date_list

In [10]:
#Anzahl der analysierten Tage:
number_of_days = len(date_list)
number_of_days

335

#### Datenqualität
Check: Missing Values (wenn status != 200, Fehler beim Download)   
**Ergebnis:** 1 fehlerhafter Download: The Economist vom 08.03.2021 wurde gelöscht  
Downloads von 6 Portalen an 335 Tagen = 2.033 Dateien 


In [11]:
# csv-Dateien auswählen
csv_data = glob.glob(os.path.join(INPUT_PATH, "*.csv"))
print(len(csv_data))  #Anzahl csv-Dateien (= Anzahl Tage): 339
csv_data[0:5]

339


['..\\input\\data-lake\\2021-03-03.csv',
 '..\\input\\data-lake\\2021-03-04.csv',
 '..\\input\\data-lake\\2021-03-05.csv',
 '..\\input\\data-lake\\2021-03-06.csv',
 '..\\input\\data-lake\\2021-03-07.csv']

In [12]:
df = [pd.read_csv(csv, index_col=0) for csv in csv_data]
csv_df = pd.concat(df)
#csv_df.file_name = csv_df.file_name.str.replace(' ', '') #Löschung Leerzeichen
#csv_df.name = csv_df.name.str.replace(' ', '') #Löschung Leerzeichen
#csv_df.head()

In [13]:
log_df = csv_df.loc[(csv_df['name'].isin(paper_list))]
#s1.str.contains('og', regex=False)
#log_df = csv_df.loc[(csv_df['name'].str.contains(paper_list, regex=True))]
log_df.head(6)  # Log-Dateien aller vorselektierten News-Portale

,name,date,file_name,status,original_url,final_url,encoding
0,sz,2021-03-03,data-lake/2021-03-03-sz.html,200,https://www.sueddeutsche.de/,https://www.sueddeutsche.de/,UTF-8
7,abendblatt,2021-03-03,data-lake/2021-03-03-abendblatt.html,200,https://www.abendblatt.de/,https://www.abendblatt.de/,UTF-8
15,handelsblatt,2021-03-03,data-lake/2021-03-03-handelsblatt.html,200,https://www.handelsblatt.com/,https://www.handelsblatt.com/,utf-8
21,economist,2021-03-03,data-lake/2021-03-03-economist .html,200,https://www.economist.com/,https://www.economist.com/,utf-8
37,cnn,2021-03-03,data-lake/2021-03-03-cnn.html,200,https://edition.cnn.com/,https://edition.cnn.com/,utf-8
38,bbc,2021-03-03,data-lake/2021-03-03-bbc.html,200,https://www.bbc.com/,https://www.bbc.com/,utf-8


In [14]:
# alle Downloads mit Fehlermeldungen anzeigen: status != 200
log_df.loc[(log_df['status'] != 200)]

,name,date,file_name,status,original_url,final_url,encoding
21,economist,2021-03-08,data-lake/2021-03-08-economist .html,403,https://www.economist.com/,https://www.economist.com/,ISO-8859-1


In [15]:
#fehlerhafte Downloads löschen
log_df = log_df.loc[(log_df['status'] == 200)]
log_df.head(6)

,name,date,file_name,status,original_url,final_url,encoding
0,sz,2021-03-03,data-lake/2021-03-03-sz.html,200,https://www.sueddeutsche.de/,https://www.sueddeutsche.de/,UTF-8
7,abendblatt,2021-03-03,data-lake/2021-03-03-abendblatt.html,200,https://www.abendblatt.de/,https://www.abendblatt.de/,UTF-8
15,handelsblatt,2021-03-03,data-lake/2021-03-03-handelsblatt.html,200,https://www.handelsblatt.com/,https://www.handelsblatt.com/,utf-8
21,economist,2021-03-03,data-lake/2021-03-03-economist .html,200,https://www.economist.com/,https://www.economist.com/,utf-8
37,cnn,2021-03-03,data-lake/2021-03-03-cnn.html,200,https://edition.cnn.com/,https://edition.cnn.com/,utf-8
38,bbc,2021-03-03,data-lake/2021-03-03-bbc.html,200,https://www.bbc.com/,https://www.bbc.com/,utf-8


In [15]:
# Rein rechnerisch sollten es 6 Zeitungen x 339 Tage = 2.034 abzügl. einem fehlerhaften Download = 2033 Dateien sein. 
# Es befinden sich jedoch nur 1980 Zeilen / Dateien im Datensatz. 
# -> Check Fehlmenge: 53 Dateien?
log_df.shape

(2033, 7)

In [17]:
# Check: Anzahl html Files im Data Lake
# Soll: 6 x 339 = 2.034

newspapers = ('*sz.html', '*handelsblatt.html',  '*abendblatt.html', '*economist .html', '*economist.html', '*cnn.html', '*bbc.html')
files_list = []
for newspaper in newspapers:
    files_list.extend(glob.glob(os.path.join(INPUT_PATH, newspaper)))
print(len(files_list))

2034


**ACHTUNG:** inkonsistente Benennung der Datei-Namen für The Economist!   
Es wurden 53 der economist-Downloads richtig benannt (*economist.html).  
Bei den anderen 286 wurde ein Leerzeichen eingefügt (*economist .html).  
**Lösung:** Beide Schreibweisen wurden in die paper_list aufgenommen, um umfangreiche Umbenennungen zu vermeiden.

## Transform 

In [18]:
# Liste aller html-Dateien erstellen
#html_data = glob.glob(os.path.join(INPUT_PATH,"*.html")
#print(len(html_data))  #Anzahl csv-Dateien 
#html_data[0:5]

### Remove HTML Formating + Stopwords

In [19]:
stopwords_url_de = "https://raw.githubusercontent.com/solariz/german_stopwords/master/german_stopwords_full.txt"
stopwords_list_de = requests.get(stopwords_url_de, allow_redirects=True).text.split("\n")[9:]

stopwords_url_en = "https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords"
stopwords_list_en = requests.get(stopwords_url_en, allow_redirects=True).text.split("\n")[9:]

In [20]:
# file_name in log_df beinhaltet die Namen aller html-Dateien, die für mich relevant sind
# Encodings: utf-8 und ISO-8859-1

#Parser Funktion:
def process_html(text):
        text = BeautifulSoup(text, "html.parser")
       
        body_text = text.body.get_text()
        #print(detect(body_text))
                
        try: 
                if "de" in detect(body_text):
                    #print("[Info] Deutsche Webpage")
                    text = text.text   
                    items = text.replace("\t", " ")
                    items = items.replace("\n", " ").lower().split(" ")
                    items = [i for i in items if len(i) > 1 and i not in stopwords_list_de]
                    return items
             
                elif "en" in detect(body_text):
                    #print("[Info] Englische Webpage")
                    text = text.text   
                    items = text.replace("\t", " ")
                    items = items.replace("\n", " ").lower().split(" ")
                    items = [i for i in items if len(i) > 1 and i not in stopwords_list_en]
                    return items
                else:
                
                    print("[ERROR] Sprache nicht korrekt erkannt.")
                
        except:
                print("[ERROR] Spracherkennung")
           

### Wordcount "Cum-Ex"

In [21]:
# Wordcount: alle html-Files, die den Begriff "Cum-Ex" enthalten (alle Varianten)
# Regex-Term = 'cum.*ex.*'

result = pd.DataFrame()

total_rows = log_df.shape[0]
print ("[START PROCESSING]")
count = 0
for i, row in log_df.iterrows():
    path = os.path.join(INPUT_BASE_PATH, row["file_name"])
    count += 1;
    
    if (count) % 100 == 0:
        print ("[PROCESSING FILE] >",count,path,"OF",total_rows)
    
    encodings = row["encoding"]
    with open(path, "r", encoding=encodings) as f:
        text = f.read()
    items = process_html(text)
    
    new = pd.Series(items).value_counts().to_frame()
    #print (new.head())
    new.columns = ["count"]
    new["word"] = new.index
    new["file_name"] = row["file_name"]
  
    new = new.loc[(new.word.astype(str).str.contains('cum.*ex.*'))]
    
   
    
    if len(new.index) > 0:
        result = pd.concat([result, new], ignore_index=True, axis=0,).reset_index(level=0, drop=True)
       
print ('[FINISHED PROCESSING]')

[START PROCESSING]
[PROCESSING FILE] > 100 ..\input\data-lake/2021-03-19-cnn.html OF 2033
[PROCESSING FILE] > 200 ..\input\data-lake/2021-04-05-handelsblatt.html OF 2033
[PROCESSING FILE] > 300 ..\input\data-lake/2021-04-22-sz.html OF 2033
[PROCESSING FILE] > 400 ..\input\data-lake/2021-05-08-cnn.html OF 2033
[PROCESSING FILE] > 500 ..\input\data-lake/2021-05-25-handelsblatt.html OF 2033
[PROCESSING FILE] > 600 ..\input\data-lake/2021-06-11-sz.html OF 2033
[PROCESSING FILE] > 700 ..\input\data-lake/2021-06-27-cnn.html OF 2033
[PROCESSING FILE] > 800 ..\input\data-lake/2021-07-14-handelsblatt.html OF 2033
[PROCESSING FILE] > 900 ..\input\data-lake/2021-07-31-sz.html OF 2033
[PROCESSING FILE] > 1000 ..\input\data-lake/2021-08-16-cnn.html OF 2033
[PROCESSING FILE] > 1100 ..\input\data-lake/2021-09-02-handelsblatt.html OF 2033
[PROCESSING FILE] > 1200 ..\input\data-lake/2021-09-19-sz.html OF 2033
[PROCESSING FILE] > 1300 ..\input\data-lake/2021-10-05-cnn.html OF 2033
[PROCESSING FILE] > 14

In [61]:
#result.head(5)

In [58]:
# Doppelpunkte entfernen
result['word'] = result['word'].str.replace(r':', '')

In [64]:
result

,count,word,file_name
0,1,cum-ex,data-lake/2021-03-03-abendblatt.html
1,1,cum-ex-affäre,data-lake/2021-03-13-abendblatt.html
2,2,cum-ex-ermittlungen,data-lake/2021-03-13-handelsblatt.html
3,1,imagecum-ex-skandal oberlandesgericht,data-lake/2021-03-13-handelsblatt.html
4,1,„cum-ex“-ausschuss,data-lake/2021-03-13-handelsblatt.html
...,...,...,...
120,1,cum-ex-prozess,data-lake/2022-01-13-sz.html
121,1,cum-ex-geschäfte,data-lake/2022-01-13-sz.html
122,2,cum-ex-skandal,data-lake/2022-01-19-sz.html
123,1,imagecum-ex-steuerskandal deutlich,data-lake/2022-01-20-handelsblatt.html


In [60]:
result.shape

(125, 3)

### Ausgabe Überschriften   
Süddeutsche Zeitung, Handelsblatt und Abendblatt 

**sz_header_df** ->  Überschriften Süddeutsche Zeitung  
**hb_header_df** ->  Überschriften Handelsblatt  
**ab_header_df** ->  Überschriften Abendblatt  

In [26]:
# Süddeutsche: Liste aller html-Dateien
sz_files = glob.glob(os.path.join(INPUT_PATH, "*sz.html"))
print(len(sz_files))  #Anzahl html-Dateien 
#sz_files[0:5]

339


In [27]:
# Handelsblatt: Liste aller html-Dateien
hb_files = glob.glob(os.path.join(INPUT_PATH, "*handelsblatt.html"))
print(len(hb_files))  #Anzahl html-Dateien 
#hb_files[0:5]

339


In [28]:
# Abendblatt: Liste aller html-Dateien
ab_files = glob.glob(os.path.join(INPUT_PATH, "*abendblatt.html"))
print(len(ab_files))  #Anzahl html-Dateien 
#ab_files[0:5]

339


In [29]:
sz_header_df = pd.DataFrame(columns = ['file_name' , 'header'])

for file in sz_files:
    with open(file, "r", encoding="utf-8") as f:
        text = f.read()
        sz_bs = BeautifulSoup(text)
        #print(len(text))
        for header in sz_bs.find_all("h3"):
            #print(header.text)
            new_row = {'file_name': 'data-lake/' + os.path.basename(file), 'header':header.text}  #kein join.os, weil file_name in den csv = data-lake/date-name.html
            sz_header_df = sz_header_df.append(new_row, ignore_index=True)
print(sz_header_df.head())       

                      file_name  \
0  data-lake/2021-03-03-sz.html   
1  data-lake/2021-03-03-sz.html   
2  data-lake/2021-03-03-sz.html   
3  data-lake/2021-03-03-sz.html   
4  data-lake/2021-03-03-sz.html   

                                              header  
0  Scholz will Corona-Beschränkungen für Geimpfte...  
1    So könnte der Fahrplan für Lockerungen aussehen  
2                              Wege aus dem Lockdown  
3                                  Der Verdachtsfall  
4  Kein Geheimdienst kann allein diese Demokratie...  


In [30]:
sz_header_df.shape

(22362, 2)

In [31]:
hb_header_df = pd.DataFrame(columns = ['file_name' , 'header'])

for file in hb_files:
    with open(file, "r", encoding="utf-8") as f:
        text = f.read()
        hb_bs = BeautifulSoup(text)
        #print(len(text))
        for header in hb_bs.select(" div.c-teaser.c-teaser--article div.c-teaser__title h3"):
            #print(header.text)
            new_row = {'file_name': 'data-lake/' + os.path.basename(file), 'header':header.text.replace("\n", "")}  
            hb_header_df = hb_header_df.append(new_row, ignore_index=True)
print(hb_header_df.head()) 

                                file_name  \
0  data-lake/2021-03-03-handelsblatt.html   
1  data-lake/2021-03-03-handelsblatt.html   
2  data-lake/2021-03-03-handelsblatt.html   
3  data-lake/2021-03-03-handelsblatt.html   
4  data-lake/2021-03-03-handelsblatt.html   

                                              header  
0  CoronakrisePandemiemüdigkeit in der Koalition:...  
1  +++ Corona-News +++BASF sieht wöchentliche Cor...  
2  ImmobilienZwei Anzeichen, dass die Bauzinsen b...  
3  KryptowährungenStartkapital 15 Euro: Was Klein...  
4  BewerbungZu teuer, zu qualifiziert, zu schräg:...  


In [32]:
hb_header_df.shape

(13458, 2)

In [33]:
ab_header_df = pd.DataFrame(columns = ['file_name' , 'header'])

for file in ab_files:
    with open(file, "r", encoding="utf-8") as f:
        text = f.read()
        ab_bs = BeautifulSoup(text)
        #print(len(text))
        for header in ab_bs.select("div.teaser__header span.teaser__headline"):
            #print(header.text)
            new_row = {'file_name': 'data-lake/' + os.path.basename(file), 'header':header.text.replace("\n", "")}  
            ab_header_df = ab_header_df.append(new_row, ignore_index=True)
print(ab_header_df.head())  

                              file_name  \
0  data-lake/2021-03-03-abendblatt.html   
1  data-lake/2021-03-03-abendblatt.html   
2  data-lake/2021-03-03-abendblatt.html   
3  data-lake/2021-03-03-abendblatt.html   
4  data-lake/2021-03-03-abendblatt.html   

                                              header  
0              Der Elbtower – nur ein Steuersparp...  
1              Corona: Merkel zu Beginn des Gipfe...  
2              Uni-Präsident entschuldigt sich fü...  
3              Warum Zierpflanzen jetzt in Hambur...  
4              Hamburg ruft nächste Gruppe zur Co...  


In [34]:
ab_header_df.shape

(19363, 2)

## Load SQL Datenbank

#### Datei: log_data (Faktentabelle)

In [35]:
log_df.to_sql(con=connection, name="log_data", if_exists="replace", index=False)

2033

#### Datei: wordcount_data (Dimension)

In [36]:
result.to_sql(con=connection, name="wordcount_data", if_exists="replace", index=False)

125

#### Datei: sz_header_df (Dimension)

In [37]:
sz_header_df.to_sql(con=connection, name="sz_header_data", if_exists="replace", index=False)

22362

#### Datei: hb_header_df (Dimension)

In [38]:
hb_header_df.to_sql(con=connection, name="hb_header_data", if_exists="replace", index=False)

13458

#### Datei: ab_header_df (Dimension)

In [39]:
ab_header_df.to_sql(con=connection, name="ab_header_data", if_exists="replace", index=False)

19363

## DWH Qualitätssicherung

#### Datei: log_data (Faktentabelle)

In [40]:
pd.read_sql_query("select * from log_data limit 5", con=connection)  # Anzeige der ersten 5 Einträge

,name,date,file_name,status,original_url,final_url,encoding
0,sz,2021-03-03,data-lake/2021-03-03-sz.html,200,https://www.sueddeutsche.de/,https://www.sueddeutsche.de/,UTF-8
1,abendblatt,2021-03-03,data-lake/2021-03-03-abendblatt.html,200,https://www.abendblatt.de/,https://www.abendblatt.de/,UTF-8
2,handelsblatt,2021-03-03,data-lake/2021-03-03-handelsblatt.html,200,https://www.handelsblatt.com/,https://www.handelsblatt.com/,utf-8
3,economist,2021-03-03,data-lake/2021-03-03-economist .html,200,https://www.economist.com/,https://www.economist.com/,utf-8
4,cnn,2021-03-03,data-lake/2021-03-03-cnn.html,200,https://edition.cnn.com/,https://edition.cnn.com/,utf-8


In [41]:
pd.read_sql_query("select count('index') from log_data", con=connection)

,count('index')
0,2033


#### Datei: wordcount_data (Dimension)

In [42]:
pd.read_sql_query("select * from wordcount_data limit 5", con=connection)  # Anzeige der ersten 5 Einträge

,count,word,file_name
0,1,cum-ex,data-lake/2021-03-03-abendblatt.html
1,1,cum-ex-affäre,data-lake/2021-03-13-abendblatt.html
2,2,cum-ex-ermittlungen,data-lake/2021-03-13-handelsblatt.html
3,1,imagecum-ex-skandal oberlandesgericht,data-lake/2021-03-13-handelsblatt.html
4,1,„cum-ex“-ausschuss,data-lake/2021-03-13-handelsblatt.html


In [43]:
pd.read_sql_query("select count('index') from wordcount_data", con=connection)

,count('index')
0,125


#### Datei: sz_header_data (Dimension)

In [44]:
pd.read_sql_query("select * from sz_header_data limit 5", con=connection)  # Anzeige der ersten 5 Einträge

,file_name,header
0,data-lake/2021-03-03-sz.html,Scholz will Corona-Beschränkungen für Geimpfte...
1,data-lake/2021-03-03-sz.html,So könnte der Fahrplan für Lockerungen aussehen
2,data-lake/2021-03-03-sz.html,Wege aus dem Lockdown
3,data-lake/2021-03-03-sz.html,Der Verdachtsfall
4,data-lake/2021-03-03-sz.html,Kein Geheimdienst kann allein diese Demokratie...


In [45]:
pd.read_sql_query("select count('index') from sz_header_data", con=connection)

,count('index')
0,22362


#### Datei: hb_header_data (Dimension)

In [46]:
pd.read_sql_query("select * from hb_header_data limit 5", con=connection)  # Anzeige der ersten 5 Einträge

,file_name,header
0,data-lake/2021-03-03-handelsblatt.html,CoronakrisePandemiemüdigkeit in der Koalition:...
1,data-lake/2021-03-03-handelsblatt.html,+++ Corona-News +++BASF sieht wöchentliche Cor...
2,data-lake/2021-03-03-handelsblatt.html,"ImmobilienZwei Anzeichen, dass die Bauzinsen b..."
3,data-lake/2021-03-03-handelsblatt.html,KryptowährungenStartkapital 15 Euro: Was Klein...
4,data-lake/2021-03-03-handelsblatt.html,"BewerbungZu teuer, zu qualifiziert, zu schräg:..."


In [47]:
pd.read_sql_query("select count('index') from hb_header_data", con=connection)

,count('index')
0,13458


#### Datei: ab_header_data (Dimension)

In [48]:
pd.read_sql_query("select * from ab_header_data limit 5", con=connection)  # Anzeige der ersten 5 Einträge

,file_name,header
0,data-lake/2021-03-03-abendblatt.html,Der Elbtower – nur ein Steuersparp...
1,data-lake/2021-03-03-abendblatt.html,Corona: Merkel zu Beginn des Gipfe...
2,data-lake/2021-03-03-abendblatt.html,Uni-Präsident entschuldigt sich fü...
3,data-lake/2021-03-03-abendblatt.html,Warum Zierpflanzen jetzt in Hambur...
4,data-lake/2021-03-03-abendblatt.html,Hamburg ruft nächste Gruppe zur Co...


In [49]:
pd.read_sql_query("select count('index') from ab_header_data", con=connection)

,count('index')
0,19363


### DWH Performance-Check

In [50]:
t1 = time()
f3 = pd.read_sql_query("select * from wordcount_data where word='cum-ex'", con=connection)
t2 = time()
t2-t1

0.003001689910888672

In [51]:
t1 = time()
f3 = pd.read_sql_query("select * from log_data where name='bbc'", con=connection)
t2 = time()
t2-t1

0.00500035285949707

In [52]:
t1 = time()
f3 = pd.read_sql_query("select * from ab_header_data where length(header) > 30", con=connection)
t2 = time()
t2-t1

0.058990478515625

## Join SQL Datenbank-Files

In [53]:
# Join log_data <> wordcount_data
query = """  
SELECT *
FROM log_data INNER JOIN wordcount_data
WHERE log_data.file_name = wordcount_data.file_name
"""
lw = pd.read_sql_query(query, con=connection)

In [54]:
# Join log_data <> sz_header_data
query = """  
SELECT *
FROM log_data INNER JOIN sz_header_data
WHERE log_data.file_name = sz_header_data.file_name
"""
lw = pd.read_sql_query(query, con=connection)

In [55]:
# Join log_data <> hb_header_data
query = """  
SELECT *
FROM log_data INNER JOIN hb_header_data
WHERE log_data.file_name = hb_header_data.file_name
"""
lw = pd.read_sql_query(query, con=connection)

In [56]:
# Join log_data <> ab_header_data
query = """  
SELECT *
FROM log_data INNER JOIN ab_header_data
WHERE log_data.file_name = ab_header_data.file_name
"""
lw = pd.read_sql_query(query, con=connection)